In [1]:
import cv2
import numpy as np

image_path = './testdata/1.png'
img = cv2.imread(image_path, 0).astype(np.float64)

In [7]:
sigma = 2**(1/4)
num_DoG_images_per_octave = 4
num_guassian_images_per_octave = num_DoG_images_per_octave + 1

first_octave = [img] + [cv2.GaussianBlur(src = img, ksize = (0, 0), sigmaX = sigma**i) for i in range(1, num_guassian_images_per_octave)]

In [11]:
# down-sample the image
DSImage = cv2.resize(first_octave[-1], 
                     (img.shape[1]//2, img.shape[0]//2), 
                     interpolation = cv2.INTER_NEAREST)

In [14]:
second_octave = [DSImage] + [cv2.GaussianBlur(src=DSImage, ksize=(0,0), sigmaX=sigma**i) for i in range(1, num_guassian_images_per_octave)]

In [18]:
gaussian_images = [first_octave, second_octave]

In [20]:
num_octaves = 2
dog_images = []

for i in range(num_octaves):
    GImg = gaussian_images[i]
    dog_img = []
    for j in range(num_DoG_images_per_octave):
        dog = cv2.subtract(GImg[j], GImg[j+1])
        dog_img.append(dog)
        M, m = max(dog.flatten()), min(dog.flatten())
        norm = (dog-m)*255/(M-m)
        cv2.imwrite(f'testdata/DoG{i+1}-{j+1}.png', norm)
    dog_images.append(dog_img)

In [47]:
dog_images[0][0].shape

(463, 360)

In [ ]:
threshold = 5.0

keypoints = []
# iterate throught each ocvate (n=2 here)
for i in range(num_octaves):
    # transform an octave into a 3-dimensional array
    dogs = np.array(dog_images[i])
    height, width = dogs[i].shape
    # examine every 3*3 cube for local extremum
    #iterate through Dog image number 1 and 2 (other than 0 and 3)
    for dog in range(1, num_DoG_images_per_octave-1):
        #iterate through every pixel
        for x in range(1, width-2):
            for y in range(1, height-2):
                pixel = dogs[dog,y,x]
                cube = dogs[dog-1:dog+2, y-1:y+2, x-1:x+2]
                # print(dog, y, x, cube.shape)
                # to check if it's local extremum
                if (np.absolute(pixel) > threshold) and ((pixel >= cube).all() or (pixel <= cube).all()):
                    keypoints.append([y*2, x*2] if i else [y, x])

In [38]:
keypoints = np.unique(np.array(keypoints), axis = 0)

In [39]:
keypoints = keypoints[np.lexsort((keypoints[:,1],keypoints[:,0]))] 

In [42]:
gt_path = './testdata/1_gt.npy'
keypoints_gt = np.load(gt_path)

if keypoints.shape[0] != keypoints_gt.shape[0]:
    print('[Error] Number of KeyPoints mismatches. Yours: %d, GT: %d'%
          (keypoints.shape[0], keypoints_gt.shape[0]))
else:
    for (x, y), (x_gt, y_gt) in zip(keypoints, keypoints_gt):
        if x != x_gt or y != y_gt:
            print('[Error] KeyPoint mismatches. Yours: (%d, %d), GT: (%d, %d)'%
                    (x, y, x_gt, y_gt))
            # return None
    print('[Info] All keypoints match.')

[Info] All keypoints match.
